In [ ]:
!pip install -q transformers datasets scikit-learn torch accelerate


In [ ]:
import numpy as np
import torch

from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score


In [ ]:
dataset = load_dataset("ag_news")
dataset


In [ ]:
dataset["train"][0]


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")


In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4
)


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()


In [ ]:
model.save_pretrained("news_bert_model")
tokenizer.save_pretrained("news_bert_model")


In [ ]:
def predict_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Move input tensors to the same device as the model
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()

    labels = ["World", "Sports", "Business", "Sci/Tech"]
    return labels[prediction]

predict_news("Apple launches a new AI powered smartphone")

In [ ]:
!pip install -q streamlit pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import BertTokenizer, BertForSequenceClassification

st.set_page_config(page_title="News Topic Classifier", layout="centered")

st.title("News Topic Classifier")
st.write("Enter a news headline and the model will predict its category.")

@st.cache_resource
def load_model():
    tokenizer = BertTokenizer.from_pretrained("news_bert_model")
    model = BertForSequenceClassification.from_pretrained("news_bert_model")
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()

labels = ["World", "Sports", "Business", "Sci/Tech"]

headline = st.text_input("News Headline")

if st.button("Predict"):
    if headline.strip() == "":
        st.warning("Please enter a headline")
    else:
        inputs = tokenizer(
            headline,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        )

        with torch.no_grad():
            outputs = model(**inputs)
            prediction = torch.argmax(outputs.logits, dim=1).item()

        st.success(f"Predicted Category: {labels[prediction]}")


In [ ]:
!ls


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print(public_url)

In [ ]:
!NGROK_AUTHTOKEN=37ONjMZwh6XJj6qK3df3aauQ694_6NYUZkdDhtaq5RinugRav python3 app.py

In [ ]:
!streamlit run app.py &>/content/logs.txt &
